In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import napari_sparrow as nas
from spatialdata import read_zarr
import os
import scanpy as sc

In [5]:
def visualize_classification(sdata,classification_name):
    # Plot the UMAP
    sc.pl.umap(sdata.table,color=[classification_name])
    # Plot the highly differential genes for each cluster
    sc.pl.rank_genes_groups(sdata.table, n_genes=8, sharey=False, show=False)
    # Plot the image with the cells
    nas.pl.plot_shapes(sdata,column=classification_name,img_layer='clahe',shapes_layer = "segmentation_mask_boundaries")